## Defining C# function as kernel function

In [6]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview"

Installed Packages Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview

In [ ]:
#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Planning.Handlebars;
using System.IO;

var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();
var kernel = Microsoft.SemanticKernel.Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(
                model,   // deployment name
                azureEndpoint, // Azure OpenAI Endpoint
                apiKey)      // Azure OpenAI Key
            .Build();

In [13]:
using Microsoft.SemanticKernel;
using System.ComponentModel;

public sealed class MyPlugin
{
    public MyPlugin()
    {

    }
    [KernelFunction, Description(@"
    Make a funny remark. 
    Example:
       The horse could not speak because its throat was sore.
       What is the point of a point?
       See shells she sells by the sea shore.")]
    public string FunnyRemark()
    {
        return "This is funny!!!";
    }    
    [KernelFunction, Description("Return a sad remark")]
    public string SadRemark()
    {
        return "This is sad, very sad!!!";
    }        
}

kernel.Plugins.Clear();
#pragma warning disable SKEXP0050
kernel.ImportPluginFromType<MyPlugin>();   

List plugins and functions

In [ ]:
    foreach (KernelFunctionMetadata func in kernel.Plugins.GetFunctionsMetadata())
    {
        Console.WriteLine($"Plugin: {func.PluginName}");
        Console.WriteLine($"   {func.Name}: {func.Description}");

        if (func.Parameters.Count > 0)
        {
            Console.WriteLine("      Params:");
            foreach (var p in func.Parameters)
            {
                Console.WriteLine($"      - {p.Name}: {p.Description}");
                Console.WriteLine($"        default: '{p.DefaultValue}'");
            }
        }

        Console.WriteLine();
    }

Now try different prompts

In [16]:
#pragma warning disable SKEXP0060
    var planner = new HandlebarsPlanner();
    var goal = "Tell me something funny, something to make me laugh. If you cannot, say 'sorry'";
    var plan = await planner.CreatePlanAsync(kernel, goal);
    var resp = await plan.InvokeAsync(kernel);
    Console.WriteLine("Plan:\n");
    Console.WriteLine(plan.ToString());
    Console.WriteLine("Result:\n");
    Console.WriteLine(resp);

Plan:

{{!-- Step 1: Call the MyPlugin-FunnyRemark helper --}}
{{set "remark" (MyPlugin-FunnyRemark)}}

{{!-- Step 2: Check if the remark is empty --}}
{{#unless (equals remark "")}}

  {{!-- Step 3: Print the funny remark --}}
  {{json (concat "Funny remark: " remark)}}

{{else}}

  {{!-- Step 4: Print "Sorry" --}}
  {{json "Sorry"}}

{{/unless}}
Result:

Funny remark: This is funny!!!


You can now modify, load and execute the plan.

In [10]:
var newSerializedPlan = @"
{{!-- Step 2: Search for Seattle weather and get the result --}}
{{set "searchResult" (search-BingSearchUrl query=searchUrl)}}

{{!-- Step 3: Print the search result to the screen --}}
{{json (concat "The weather in Seattle is: " searchResult)}}
";
var newPlan = new HandlebarsPlan(serializedPlan);
Console.WriteLine(await newPlan.InvokeAsync(kernel);)

Error: (3,10): error CS1002: ; expected
(3,22): error CS1002: ; expected
(3,22): error CS1010: Newline in constant
(3,72): error CS1002: ; expected
(5,14): error CS1002: ; expected
(5,15): error CS1002: ; expected
(5,15): error CS1513: } expected
(5,27): error CS1002: ; expected
(5,41): error CS1002: ; expected
(5,48): error CS1002: ; expected
(5,57): error CS1002: ; expected
(6,18): error CS1003: Syntax error, ',' expected
(6,48): error CS1003: Syntax error, ',' expected
(6,61): error CS1002: ; expected
(7,1): error CS1010: Newline in constant
(7,3): error CS1002: ; expected
(9,52): error CS1026: ) expected
(9,53): error CS7017: Member definition, statement, or end-of-file expected